In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import all neccesary imports:

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

Read the data into pandas:

In [ ]:
df = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding = 'latin-1')
df.head()

Perform data exploratory to get to know the data:

In [ ]:
df.head()

In [ ]:
df.describe()

Check the percentage of missing values in the dataset:

In [ ]:
(df.isna().sum() * 100) / df.shape[0]

It can be seen for v1 and v2 data have no missing values while the rest of the columns have less than 1% of data existed. For the sake of simplicity and ease of understanding for the project, only the v1 (label) and v2 (message) will be used.

In [ ]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
df = df.rename(columns = {'v1':'label', 'v2':'message'})

Now the remaining data looks as below:

In [ ]:
df.describe()

Additional feature that can be extracted from the message is the length of the text, which can be added to the data:

In [ ]:
df['length'] = df['message'].apply(len)
df.head()

Text processing and vectorizing into tokens:

In [ ]:
text_feat = df['message'].copy()

Remove punctuations

In [ ]:
def text_process(text):
    text = text.translate(str.maketrans('','', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [ ]:
text_feat = text_feat.apply(text_process)

In [ ]:
vectorizer = TfidfVectorizer("english")

In [ ]:
features = vectorizer.fit_transform(text_feat)

**Model classifiers and predictions**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df['label'], test_size = 0.2, random_state = 16)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [ ]:
clfs = {'SVC' : svc,'KN' : knc, 'NB': mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'BgC': bc, 'ETC': etc}

In [ ]:
def train_classifier(clf, X_train, y_train):    
    clf.fit(X_train, y_train)

In [ ]:
def predict_labels(clf, features):
    return (clf.predict(features))

In [ ]:
pred_scores = []
for k,v in clfs.items():
    train_classifier(v, X_train, y_train)
    pred = predict_labels(v,X_test)
    pred_scores.append((k, [accuracy_score(y_test,pred)]))

In [ ]:
pred_scores

Best accuracy achieves by Naive Bayes with 0.9847%